In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.models import load_model

In [ ]:
dataset_train = pd.read_csv("train_data.csv")
Date = dataset_train.iloc[:,1:2].values
Time = dataset_train.iloc[:,2:3].values
MidPrice = dataset_train.iloc[:,3:4].values
LastPrice = dataset_train.iloc[:,4:5].values
Volume = dataset_train.iloc[:,5:6].values
BidPrice1 = dataset_train.iloc[:,6:7].values
BidVolume1 = dataset_train.iloc[:,7:8].values
AskPrice1 = dataset_train.iloc[:,8:9].values
AskVolume1 = dataset_train.iloc[:,9:10].values

In [ ]:
DVolume = Volume.copy()
for i in range(1,len(Volume)):
    DVolume[i] = Volume[i]- Volume[i-1]
DVolume[0] = DVolume[1]
DBidVolume1 = BidVolume1.copy()
for i in range(1,len(BidVolume1)):
    DBidVolume1[i] = BidVolume1[i] - BidVolume1[i-1]
DBidVolume1[0] = DBidVolume1[1]
DAskVolume1 = AskVolume1.copy()
for i in range(1,len(AskVolume1)):
    DAskVolume1[i] = AskVolume1[i] - AskVolume1[i-1]
DAskVolume1[0] = DAskVolume1[1]


In [ ]:
RTime = Time.copy()
timeArray = []
for i in range(len(Time)):
    time_s = Date[i][0] + ' ' + Time[i][0]
    timeArray.append(time.strptime(time_s,'%Y-%m-%d %H:%M:%S'))
print(timeArray)
for i in range(1,len(Time)):
    RTime[i] = int(time.mktime(timeArray[i]))
RTime[0] = RTime[1]-3
RTime

In [ ]:
def zscore(arr):
    u = arr.mean()
    sig = arr.std()
    res = np.copy(arr)
    for i in range(len(arr)):
        res[i] = (arr[i]-u)/sig
    return res,u,sig

def rev_zscore(arr,u,sig):
    res = np.copy(arr)
    for i in range(len(arr)):
        res[i] = arr[i]*sig + u
    return res

In [ ]:
LastPrice_scaled,LPu,LPsig = zscore(LastPrice)
BidPrice1_scaled,BP1u,BP1sig = zscore(BidPrice1)
AskPrice1_scaled,AP1u,AP1sig = zscore(AskPrice1)
DVolume_scaled,DVu,DVsig = zscore(DVolume)
DBidVolume1_scaled,BV1u,BV1sig = zscore(DBidVolume1)
DAskVolume1_scaled,AV1u,AV1sig = zscore(DAskVolume1)
MidPrice_scaled,MPu,MPsig = zscore(MidPrice)

In [ ]:
X_train = []
y_train = []

for i in range(10,len(DVolume_scaled)-20,4):
    if RTime[i+20] - RTime[i] != 60:
        continue
    tem_loader = np.array([])
    tem_loader = np.append(tem_loader,LastPrice_scaled[i-10:i,0])
    tem_loader = np.append(tem_loader,DVolume_scaled[i-10:i,0])
    tem_loader = np.append(tem_loader,DBidVolume1_scaled[i-10:i,0])
    tem_loader = np.append(tem_loader,DAskVolume1_scaled[i-10:i,0])
    tem_loader = np.append(tem_loader,BidPrice1_scaled[i-10:i,0])
    tem_loader = np.append(tem_loader,AskPrice1_scaled[i-10:i,0])
    X_train.append(tem_loader)
    y_train.append(MidPrice[i:i+20,0].mean()-MidPrice[i-1])
X_train,y_train = np.array(X_train),np.array(y_train)

In [ ]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
y_train,ytu,ytsig = zscore(y_train)

In [ ]:
X_train_cnn = np.reshape(X_train,(X_train.shape[0],10,6,1))

In [ ]:
regressor_cnn = Sequential()
regressor_cnn.add(Conv2D(filters=32,kernel_size = (3,3),strides=(1,1),padding='same',input_shape=(X_train_cnn.shape[1],X_train_cnn.shape[2],1),activation='relu'))
regressor_cnn.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
y_2nd = np.array(y_2nd)
y_2nd.shape
y_2nd = np.reshape(y_2nd, (79312, 15, 32))
print(y_2nd.shape)
print(X_train.shape)
X_train = np.reshape(X_train,(-1,15,4))
X_train.shape
X_train = np.concatenate((X_train,y_2nd),axis=2)
X_train.shape

In [ ]:
regressor_lstm = Sequential()
regressor_lstm.add(LSTM(units=128,return_sequences=True,input_shape=(X_train.shape[1],X_train.shape[2])))
regressor_lstm.add(Dropout(0.15))
regressor_lstm.add(LSTM(units=128,return_sequences=True))
regressor_lstm.add(Dropout(0.15))
regressor_lstm.add(LSTM(units=128))
regressor_lstm.add(Dropout(0.2))
regressor_lstm.add(Dense(128,activation='relu'))
regressor_lstm.add(Dense(128,activation='relu'))
regressor_lstm.add(Dense(units=1))
regressor_lstm.compile(optimizer = 'Adam',loss='mean_squared_error')
regressor_lstm.fit(X_train,y_train,epochs=50,batch_size=32)

In [ ]:
dataset_test = pd.read_csv('test_data.csv')
MidPrice_test = dataset_test.iloc[:,3:4].values
LastPrice_test = dataset_test.iloc[:,4:5].values
Volume_test = dataset_test.iloc[:,5:6].values
BidPrice1_test = dataset_test.iloc[:,6:7].values
BidVolume1_test = dataset_test.iloc[:,7:8].values
AskPrice1_test = dataset_test.iloc[:,8:9].values
AskVolume1_test = dataset_test.iloc[:,9:10].values

In [ ]:
DVolume_test = Volume_test.copy()
for i in range(1,len(Volume_test)):
    DVolume_test[i] = Volume_test[i]- Volume_test[i-1]
DVolume_test[0] = DVolume_test[1]

DBidVolume1_test = BidVolume1_test.copy()
for i in range(1,len(BidVolume1_test)):
    DBidVolume1_test[i] = BidVolume1_test[i] - BidVolume1_test[i-1]
DBidVolume1_test[0] = DBidVolume1_test[1]
DAskVolume1_test = AskVolume1_test.copy()
for i in range(1,len(AskVolume1_test)):
    DAskVolume1_test[i] = AskVolume1_test[i] - AskVolume1_test[i-1]
DAskVolume1_test[0] = DAskVolume1_test[1]

In [ ]:
LastPrice_test_scaled,LPu,LPsig = zscore(LastPrice_test)
BidPrice1_test_scaled,BP1u,BP1sig = zscore(BidPrice1_test)
AskPrice1_test_scaled,AP1u,AP1sig = zscore(AskPrice1_test)
DVolume_test_scaled,DVu,DVsig = zscore(DVolume_test)
DBidVolume1_test_scaled,BV1u,BV1sig = zscore(DBidVolume1_test)
DAskVolume1_test_scaled,AV1u,AV1sig = zscore(DAskVolume1_test)
MidPrice_test_scaled,MPu,MPsig = zscore(MidPrice_test)

In [ ]:
X_test = []
y_rec = []
for i in range(len(MidPrice_test) // 10):
    tem_loader = np.array([])
    tem_loader = np.append(tem_loader,LastPrice_test_scaled[i*10:(i+1)*10])
    tem_loader = np.append(tem_loader,DVolume_test_scaled[i*10:(i+1)*10])
    tem_loader = np.append(tem_loader,DBidVolume1_test_scaled[i*10:(i+1)*10])
    tem_loader = np.append(tem_loader,DAskVolume1_test_scaled[i*10:(i+1)*10])
    tem_loader = np.append(tem_loader,BidPrice1_test_scaled[i*10:(i+1)*10])
    tem_loader = np.append(tem_loader,AskPrice1_test_scaled[i*10:(i+1)*10])
    X_test.append(tem_loader)
    print(MidPrice_test[i*10+9])
    y_rec.append(MidPrice_test[i*10+9])

X_test = np.array(X_test)

In [ ]:
X_test_cnn = np.reshape(X_test,(X_test.shape[0],10,6,1))
y_2nd_test = regressor_cnn.predict(X_test_cnn)
y_2nd_test = np.reshape(y_2nd_test, (1000, 15, 32))
X_test_cnn = np.reshape(X_test_cnn,(-1,15,4))

X_test = np.concatenate((X_test_cnn,y_2nd_test),axis=2)
y_test = regressor_lstm.predict(X_test)
y_test = rev_zscore(y_test,ytu,ytsig)

y_test = y_rec+y_test
y_test

In [ ]:
num_id = []
mid_price = []
for i,j in enumerate(y_test):
    num_id.append(i)
    mid_price.append(j[0])
datafm = pd.DataFrame({'mid_price':mid_price})
print(datafm)
datafm.to_csv('res0.csv',index = False,sep = ' ')